In [ ]:
import json
import pandas as pd
import spacy

In [3]:
# Charger les fichiers
with open("transcriptions_ordonnées.json", "r", encoding="utf-8") as f:
    transcriptions_data = json.load(f)

df_erreurs = pd.read_csv("erreurs_déclinaison_corrigé.csv", sep=',', encoding='utf-8', header=0)
df_erreurs.columns = df_erreurs.columns.str.strip()

df_stat = pd.read_csv("statistiques_erreurs_par_apprenant.csv", sep=',', encoding='utf-8', header=0)

# Préparer le dictionnaire des transcriptions par apprenant
transcriptions_dict = {}
for entry in transcriptions_data:
    if "transcription automatique" not in entry:
        continue
    num_apprenant = entry.get("numéro d'apprenant", "")
    pays = entry.get("pays", "")
    transcription_auto = entry["transcription automatique"]
    transcription_man = []
    transcription_corr = []
    for k, v in entry.items():
        if k.startswith("transcription manuelle"):
            transcription_man.append(v)
        elif k.startswith("version correcte en polonais"):
            transcription_corr.append(v)
    transcriptions_dict[num_apprenant] = {
        "pays": pays,
        "trans_auto": transcription_auto,
        "trans_man": " ".join(transcription_man),
        "trans_corr": " ".join(transcription_corr)
    }

# Charger le modèle spaCy pour le polonais
nlp = spacy.load("pl_core_news_sm")

# Générer les lignes
rows = []
for idx, row in df_stat.iterrows():
    mot_correct = row["Forme correcte"]
    mot_interlangue = row["Forme erronée (manuel)"]

    apprenants_list = [a.strip() for a in str(row["Apprenants"]).split(",")]

    for apprenant in apprenants_list:
        if apprenant not in transcriptions_dict:
            continue

        doc = nlp(mot_correct)
        if not any(token.pos_ in ["NOUN", "ADJ"] for token in doc):
            continue

        match_incorrecte = df_erreurs[df_erreurs["forme incorrecte"] == mot_interlangue]
        if match_incorrecte.empty:
            cas = ""
            correspondance = ""
        else:
            cas = match_incorrecte.iloc[0]["cas"]
            correspondance = match_incorrecte.iloc[0]["correspondance"]

        match_correcte = df_erreurs[df_erreurs["forme correcte"] == mot_correct]
        if match_correcte.empty:
            cas_correct = ""
        else:
            cas_correct = match_correcte.iloc[0]["cas correct"]

        trans_data = transcriptions_dict[apprenant]
        pays = trans_data["pays"]
        trans_auto = trans_data["trans_auto"]
        trans_man = trans_data["trans_man"]
        trans_corr = trans_data["trans_corr"]

        first_row = not any(r["apprenant"] == apprenant for r in rows)

        rows.append({
            "pays": pays,
            "apprenant": apprenant,
            "mot (interlangue)": mot_interlangue,
            "cas": cas,
            "mot (correcte en polonais)": mot_correct,
            "cas correct": cas_correct,
            "mot (automatique)": "",
            "transcription automatique": trans_auto if first_row else "",
            "transcription manuelle": trans_man if first_row else "",
            "version correcte en polonais": trans_corr if first_row else "",
            "correspondance": correspondance
        })


df_result = pd.DataFrame(rows)
df_result['apprenant'] = df_result['apprenant'].astype(int)

def has_transcriptions(row):
    if row['transcription automatique'] and row['transcription manuelle'] and row['version correcte en polonais']:
        return 0
    else:
        return 1

df_result['ordre'] = df_result.apply(has_transcriptions, axis=1)
df_result = df_result.sort_values(by=['apprenant', 'ordre'])
df_result = df_result.drop(columns=['ordre'])

df_result.to_csv("interlanguage_déclinaison.csv", index=False, encoding="utf-8")

print("Le fichier interlanguage_déclinaison.csv a été créé avec succès.")


Le fichier interlanguage_déclinaison.csv a été créé avec succès.


##### Correction du fichier `interlanguage_déclinaison.csv` - ajout des mots dans la colonne `mot (automatique)` -> `interlanguage_déclinaison_corrigé.csv`

In [11]:
# Charger le fichier CSV
df = pd.read_csv("interlanguage_déclinaison_corrigé.csv", encoding="utf-8")

# Supprimer les lignes où "mot (automatique)" est vide ou manquant
df = df[df["mot (automatique)"].notna() & (df["mot (automatique)"].str.strip() != "")]

# Créer les dictionnaires pour recherche rapide
dict_interlangue = dict(zip(df["mot (interlangue)"], df["cas (interlangue)"]))
dict_correct = dict(zip(df["mot (correcte en polonais)"], df["cas correct"]))

# Construire la liste des valeurs pour "cas (mot automatique)"
cas_auto_list = []
nb_spacy = 0       # Compteur pour cas détectés via spaCy
nb_a_completer = 0 # Compteur pour "à compléter"

for mot_auto in df["mot (automatique)"].astype(str).str.strip():
    if mot_auto == "-":
        cas_auto_list.append("-")
        continue

    if mot_auto in dict_interlangue:
        cas_auto_list.append(dict_interlangue[mot_auto])
    elif mot_auto in dict_correct:
        cas_auto_list.append(dict_correct[mot_auto])
    else:
        # Utiliser spaCy pour deviner le cas grammatical
        doc = nlp(mot_auto)
        if doc and doc[0].morph.get("Case"):
            cas_detecte = doc[0].morph.get("Case")[0]  # Ex: "Acc", "Nom", "Gen"
            cas_auto_list.append(f"{cas_detecte} (spaCy)")
            nb_spacy += 1
        else:
            cas_auto_list.append("à compléter")
            nb_a_completer += 1

# Insérer la nouvelle colonne juste après "mot (automatique)"
insert_pos = df.columns.get_loc("mot (automatique)") + 1
df.insert(insert_pos, "cas (mot automatique)", cas_auto_list)

# Supprimer la colonne "correspondance"
if "correspondance" in df.columns:
    df = df.drop(columns=["correspondance"])

# Sauvegarder le fichier mis à jour
df.to_csv("interlanguage_déclinaison_corrigé.csv", index=False, encoding="utf-8")

# Afficher les statistiques
print(f"Cas détectés par spaCy : {nb_spacy}")
print(f"Nombre d'entrées 'à compléter' : {nb_a_completer}")
print("Le fichier a été mis à jour.")


Cas détectés par spaCy : 59
Nombre d'entrées 'à compléter' : 15
Le fichier a été mis à jour.


##### Correction du fichier `interlanguage_déclinaison_corrigé.csv` - correction des cas dans la colonne `cas (mot automatique)`

In [12]:
# Suppression des lignes où le cas interlangue est correct (c'est-à-dire cas (interlangue) == cas correct)
df_decl_err = df[df["cas (interlangue)"] != df["cas correct"]]

df_decl_err.to_csv("interlanguage_déclinaison_corrigé_final.csv", index=False, encoding="utf-8")
